In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [2]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')
test_day.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
27011,2025-01-13,52.000000,52.310001,50.869999,51.509998,51.509998,30845300,VALE3.SA,-0.010002,51.887704,NaN,51.509998,NaN,NaN,NaN,0.010002,0.000000,22.087279,50.869999,55.400002,22.465881,-77.534119,-1.594897,-1.362351,-0.056011,124437362
27012,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,51.876392,NaN,51.849998,NaN,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,22.216176,-77.783824,-1.614043,-1.412690,-0.054067,103058362
27013,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,52.093474,NaN,52.599998,NaN,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,27.008267,-72.991733,-1.593333,-1.448818,-0.047025,138196062
27014,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,52.266431,NaN,52.669998,NaN,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,30.826305,-69.173695,-1.545152,-1.468085,-0.036330,358458862
27015,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,52.933502,NaN,54.490002,NaN,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,47.766288,-52.233712,-1.436582,-1.461784,-0.014894,413117562


In [3]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [4]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5740,2025-01-13,11.18,11.31,11.12,11.24,11.24,28945300,ABEV3.SA,0.08,11.354808,NaN,11.24,NaN,NaN,NaN,0.00,0.08,26.196436,11.08,12.91,15.016841,-84.983159,-0.441777,-0.309671,-0.080445,-2991616373
5741,2025-01-14,11.21,11.37,11.15,11.21,11.21,42625300,ABEV3.SA,-0.03,11.311366,NaN,11.21,NaN,NaN,NaN,0.03,0.00,25.477876,11.08,12.34,18.362358,-81.637642,-0.453993,-0.338535,-0.070226,-3034241673
5742,2025-01-15,11.30,11.43,11.16,11.39,11.39,45380700,ABEV3.SA,0.18,11.334956,NaN,11.39,NaN,NaN,NaN,0.00,0.18,37.371097,11.08,12.34,20.234612,-79.765388,-0.456509,-0.362130,-0.055105,-2988860973
5743,2025-01-16,11.42,11.42,11.04,11.08,11.08,29298500,ABEV3.SA,-0.31,11.258469,NaN,11.08,NaN,NaN,NaN,0.31,0.00,28.372909,11.04,12.24,18.205775,-81.794225,-0.459378,-0.381580,-0.045585,-3018159473
5744,2025-01-17,10.84,11.09,10.77,11.09,11.09,51433600,ABEV3.SA,0.01,11.207929,NaN,11.09,NaN,NaN,NaN,0.00,0.01,29.009143,10.77,12.19,30.840023,-69.159977,-0.460423,-0.397348,-0.037478,-3069593073


In [5]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2024-12-05'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.tail(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5730,2024-12-26,12.18,12.24,11.99,12.14,12.14,21166800,ABEV3.SA,-0.080000,12.467370,NaN,12.14,-1.0,11.24,11.354808,0.080000,0.00,25.409653,11.99,14.53,18.794106,-81.205894,-0.027737,0.108392,-0.107626,-2630572973
5731,2024-12-27,12.16,12.19,11.97,12.07,12.07,37624000,ABEV3.SA,-0.070001,12.348159,NaN,12.07,-1.0,11.21,11.311366,0.070001,0.00,24.226632,11.97,14.53,14.771826,-85.228174,-0.080376,0.070639,-0.099313,-2668196973
5732,2024-12-30,12.06,12.16,11.74,11.74,11.74,38907400,ABEV3.SA,-0.330000,12.165711,NaN,11.74,-1.0,11.39,11.334956,0.330000,0.00,19.331009,11.74,14.50,15.424325,-84.575675,-0.135255,0.029460,-0.100223,-2707104373
5733,2025-01-02,11.72,11.89,11.49,11.60,11.60,33417900,ABEV3.SA,-0.139999,11.995998,NaN,11.60,-1.0,11.08,11.258469,0.139999,0.00,17.590948,11.49,14.50,16.810569,-83.189431,-0.190249,-0.014482,-0.099167,-2740522273
5734,2025-01-03,11.49,11.55,11.27,11.33,11.33,41365900,ABEV3.SA,-0.270000,11.796199,NaN,11.33,-1.0,11.09,11.207929,0.270000,0.00,14.655374,11.27,14.36,17.029069,-82.970931,-0.247106,-0.061007,-0.109828,-2781888173
5735,2025-01-06,11.36,11.50,11.21,11.29,11.29,50881500,ABEV3.SA,-0.040000,11.644339,NaN,11.29,NaN,NaN,NaN,0.040000,0.00,14.248904,11.21,13.58,18.326538,-81.673462,-0.300950,-0.108995,-0.111398,-2832769673
5736,2025-01-07,11.30,11.58,11.27,11.54,11.54,34905200,ABEV3.SA,0.250000,11.613037,NaN,11.54,NaN,NaN,NaN,0.000000,0.25,28.541568,11.21,13.42,18.236978,-81.763022,-0.342203,-0.155637,-0.107162,-2867674873
5737,2025-01-08,11.48,11.53,11.28,11.42,11.42,39248800,ABEV3.SA,-0.120000,11.555126,NaN,11.42,NaN,NaN,NaN,0.120000,0.00,26.129481,11.21,13.42,15.616564,-84.383436,-0.375243,-0.199558,-0.095405,-2906923673
5738,2025-01-09,11.43,11.51,11.30,11.40,11.40,24470100,ABEV3.SA,-0.020000,11.508588,NaN,11.40,NaN,NaN,NaN,0.020000,0.00,25.711601,11.21,13.19,15.080212,-84.919788,-0.400566,-0.239760,-0.087176,-2931393773
5739,2025-01-10,11.40,11.43,11.08,11.16,11.16,31277300,ABEV3.SA,-0.240000,11.404012,NaN,11.16,NaN,NaN,NaN,0.240000,0.00,21.050384,11.08,13.08,16.200587,-83.799413,-0.424183,-0.276644,-0.085291,-2962671073


##### Carregar modelo aprendido geral

In [15]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_D10_0.99.pkl')

#### Carregar modelo aprendido especifico.

#### Ambev

In [17]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Ambev.pkl')

#### Itau

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Itau = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Itau.pkl')

#### Magalu

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Magalu = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Magalu.pkl')

#### Petrobrás

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Petro = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Petro.pkl')

#### Vale

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Vale = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Vale.pkl')

In [6]:
import pandas as pd
import numpy as np
import joblib

# Carregar o DataFrame
test_day = pd.read_parquet('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')

# Converter a coluna de datas para datetime
test_day['Date'] = pd.to_datetime(test_day['Date'])

# Definir a data inicial para filtrar
inicio = '2024-12-05'

# Lista de tickers e modelos correspondentes
tickers_models = {
    'ABEV3.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Ambev.pkl',
    'ITUB4.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Itau.pkl',
    'MGLU3.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Magalu.pkl',
    'PETR4.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Petro.pkl',
    'VALE3.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Vale.pkl',
}

# Resultado final armazenado em um DataFrame
final_predictions = []

# Loop para processar cada Ticker
for ticker, model_path in tickers_models.items():
    print(f"Processando {ticker}...")
    
    # Filtrar o DataFrame para o Ticker atual
    test_day_ticker = test_day[test_day['Ticker'] == ticker].reset_index(drop=True)
    
    # Filtrar as datas a partir da data inicial
    test_day_ticker = test_day_ticker[test_day_ticker['Date'] >= inicio]
    
    # Verificar se há dados suficientes
    if test_day_ticker.empty:
        print(f"  Nenhum dado disponível para {ticker} após {inicio}.")
        continue
    
    # Selecionar as features para o modelo
    Features = test_day_ticker[['RSI', 'k_percent', 'r_percent', 'Price_Rate_Of_Change', 'MACD', 'MACD_EMA', 'On Balance Volume']].to_numpy()
    
    # Carregar o modelo correspondente
    model = joblib.load(model_path)
    
    # Fazer previsões
    predictions = model.predict(Features)
    
    # Adicionar as previsões ao DataFrame
    test_day_ticker['Prediction'] = predictions
    
    # Armazenar os resultados no conjunto final
    final_predictions.append(test_day_ticker)

# Concatenar todos os resultados em um único DataFrame
final_result_df = pd.concat(final_predictions, ignore_index=True)

final_result_df.tail()



Processando ABEV3.SA...
Processando ITUB4.SA...


c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Processando MGLU3.SA...
Processando PETR4.SA...


c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Processando VALE3.SA...


c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
135,2025-01-13,52.000000,52.310001,50.869999,51.509998,51.509998,30845300,VALE3.SA,-0.010002,51.887704,1.0,51.509998,NaN,NaN,NaN,0.010002,0.000000,22.087279,50.869999,55.400002,22.465881,-77.534119,-1.594897,-1.362351,-0.056011,124437362
136,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,51.876392,1.0,51.849998,NaN,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,22.216176,-77.783824,-1.614043,-1.412690,-0.054067,103058362
137,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,52.093474,1.0,52.599998,NaN,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,27.008267,-72.991733,-1.593333,-1.448818,-0.047025,138196062
138,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,52.266431,1.0,52.669998,NaN,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,30.826305,-69.173695,-1.545152,-1.468085,-0.036330,358458862
139,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,52.933502,1.0,54.490002,NaN,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,47.766288,-52.233712,-1.436582,-1.461784,-0.014894,413117562


In [7]:
final_result_df['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [8]:
final_result_df.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day_all.xlsx', index=False)

In [9]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(final_result_df.shape[0], final_result_df.shape[1]))

# Any row that has a `NaN` value will be dropped.
final_result_df = final_result_df.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(final_result_df.shape[0], final_result_df.shape[1]))


Antes de tirar as NaN nós tinhamos 140 linhas e 26 colunas
Depois da retirada dos NaN temos 90 linhas e 26 colunas


In [10]:
# Comparar valores e calcular a precisão
final_result_df['Match'] = (final_result_df['Prediction'] == final_result_df['Verify']).astype(int)
accuracy = final_result_df['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 74.44%


#### Uma variavel.

In [ ]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
5717,86.722945,51.583477,-48.416523,0.066920,0.052492,-0.040186,-2122382047
5718,77.627058,57.280530,-42.719470,0.080322,0.106557,-0.010837,-2075932847
5719,77.160149,63.017409,-36.982591,0.089671,0.158135,0.022957,-2050611047
5720,78.073184,68.133232,-31.866768,0.099141,0.206047,0.059575,-2027668547
5721,57.351316,68.103080,-31.896920,0.101865,0.241183,0.095897,-2076819147


In [ ]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

test_day_AMBEV.tail(30)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5717,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,13.389995,-1.0,14.42,-1.0,12.78,13.137559,0.000000,0.49,86.722945,12.25,14.46,51.583477,-48.416523,0.052492,-0.040186,0.066920,-2122382047
5718,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,13.555996,-1.0,14.22,-1.0,12.23,12.956047,0.200000,0.00,77.627058,12.25,14.53,57.280530,-42.719470,0.106557,-0.010837,0.080322,-2075932847
5719,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.686797,-1.0,14.21,-1.0,12.22,12.808838,0.010000,0.00,77.160149,12.25,14.53,63.017409,-36.982591,0.158135,0.022957,0.089671,-2050611047
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.803438,1.0,14.27,-1.0,12.14,12.675070,0.000000,0.06,78.073184,12.25,14.53,68.133232,-31.866768,0.206047,0.059575,0.099141,-2027668547
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.802750,-1.0,13.80,-1.0,12.07,12.554056,0.470000,0.00,57.351316,12.25,14.53,68.103080,-31.896920,0.241183,0.095897,0.101865,-2076819147
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.662200,1.0,13.10,-1.0,11.74,12.391245,0.700000,0.00,39.386518,12.25,14.53,61.938608,-38.061392,0.254750,0.127667,0.087117,-2126436047
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.545760,1.0,13.08,-1.0,11.60,12.232996,0.020000,0.00,38.983936,12.25,14.53,56.831589,-43.168411,0.253188,0.152772,0.066268,-2157303947
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.404608,1.0,12.84,-1.0,11.33,12.052397,0.240000,0.00,34.150823,12.25,14.53,50.640711,-49.359289,0.237819,0.169781,0.036456,-2189399447
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.343687,1.0,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,47.968714,-52.031286,0.218208,0.179466,0.016082,-2228963947
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.226949,1.0,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,42.848658,-57.151342,0.191043,0.181782,-0.012177,-2269828147


In [ ]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)

In [ ]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))

# Any row that has a `NaN` value will be dropped.
test_day_AMBEV = test_day_AMBEV.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))


Antes de tirar as NaN nós tinhamos 16 linhas e 26 colunas
Depois da retirada dos NaN temos 6 linhas e 26 colunas


In [ ]:
# Comparar valores e calcular a precisão
test_day_AMBEV['Match'] = (test_day_AMBEV['Prediction'] == test_day_AMBEV['Verify']).astype(int)
accuracy = test_day_AMBEV['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 83.33%
